# Cahn-Hilliard Equation using Flory-Huggins

This is the implementation for Cahn-Hilliard Equation for two phase separation in 2D using FEniCSx.
The boundary-value problem is described by:
$$\frac{\partial c}{\partial t} - \nabla \cdot M \left( \nabla \left( \frac{\partial f}{\partial c} - \frac{\epsilon^2}{2} \Delta c \right) \right) = 0 \quad \in \Omega$$

We use split formulation to rephrase the fourth-order equation into two coupled second-order equations:
$$
\begin{aligned}
\frac{\partial c}{\partial t} - \nabla \cdot M \nabla \mu &= 0 \quad \in \Omega\\
\mu - \frac{\partial f}{\partial c} + \frac{\epsilon^2}{2} \Delta c &= 0 \quad \in \Omega
\end{aligned}
$$

## Variational formulation

### Weak formulation

The variational form of the equations are:
$$
\begin{aligned}
\int_\Omega \frac{\partial c}{\partial t} q dx + \int_\Omega M \nabla \mu \cdot \nabla q dx = 0 \\
\int_\Omega \mu v dx - \int_\Omega \frac{\partial f}{\partial c} v dx - \int_\Omega \frac{\epsilon^2}{2} \nabla c \cdot \nabla v dx = 0
\end{aligned}
$$

### Crank-Nicholson time stepper

The sampling of the first PDE at time $t_{n+1}$ is given by:
$$
\begin{aligned}
\int_\Omega \frac{c_{n+1} - c_n}{dt} q dx + \frac{1}{2} \int_\Omega M \nabla \left( \mu_{n+1} + \mu_n \right) \cdot \nabla q dx = 0 \\
\int_\Omega \mu_{n+1} v dx - \int_\Omega \frac{d f_{n+1}}{dc} v dx - \int_\Omega \frac{\epsilon^2}{2} \nabla c_{n+1} \cdot \nabla v dx = 0
\end{aligned}
$$

## Problem definition

- Domain: $\Omega = [0,1] \times [0,1]$
- Local energy: $f = \frac{R T}{V} \left[ \frac{\phi_1}{N_1} \ln \phi_1 + \frac{\phi_2}{N_2} \ln \phi_2 + \chi_{1,2} \phi_1 \phi_2 \right]$
- Gradient energy coefficent: $\lambda = 1 \times 10^{-2}$
- Mobility coefficient: $M = 1$
- Flory-Huggins parameter: $\chi_{1,2} = 3$
- Length of polymer chains: $N_1 = N_2 = 1$

In [6]:
# Import Libraries
from mpi4py import MPI
from dolfinx import mesh, fem, io, log, default_real_type
import numpy as np
import ufl
from basix.ufl import element, mixed_element
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from petsc4py import PETSc
from pathlib import Path

In [7]:
# Define simulation parameters
J2eV = 1 # convert to eV
R = 1.0
T = 1.0
V = 1.0
epsilon_ = 0.02
lambda_ = epsilon_
M = 1.0
chi = 2.0
N1 = 5.0
N2 = 5.0
dt = 5.0e-05
T = 1.0e-03 # End time
num_steps = T / dt # Number of time steps
# Print properties
print(f"M={M}, V={V}, R={R}, lambda={lambda_}")

# Create mesh
domain = mesh.create_rectangle(MPI.COMM_WORLD, [np.array([0, 0]), np.array([2, 1])], [512, 256], mesh.CellType.quadrilateral)

# Create FunctionSpace
P1 = element("Lagrange", domain.basix_cell(), 1, dtype=default_real_type)
ME = fem.functionspace(domain, mixed_element([P1, P1]))

# Apply Dirichlet BCs
# Identify the facets on the left and right boundaries
facets = mesh.locate_entities_boundary(domain, domain.topology.dim - 1, lambda x: np.full(x.shape[1], True, dtype=bool))

# Define variational problem
# Define trial and test functions
u = fem.Function(ME)
# Previous solution
u0 = fem.Function(ME)
q, v = ufl.TestFunctions(ME)
# Split mixed functions
c, mu = ufl.split(u)
c0, mu0 = ufl.split(u0)

# Initial condition
u.x.array[:] = 0.0
rng = np.random.default_rng(2)
u.sub(0).interpolate(lambda x: 0.5 + 0.02 * (0.5 - rng.random(x.shape[1])))
u.x.scatter_forward()

# Compute df/dc
c = ufl.variable(c)
f = (R * T / V) * (c * ufl.ln(c) / N1 + (1 - c) * ufl.ln(1 - c) / N2 + chi * c * (1 - c))
dfdc = ufl.diff(f, c)

# Define residuals
F0 = ufl.inner(c, q) * ufl.dx - ufl.inner(c0, q) * ufl.dx + (dt/2) * ufl.inner(ufl.grad(mu + mu0), ufl.grad(q)) * ufl.dx
F1 = ufl.inner(mu, v) * ufl.dx - ufl.inner(dfdc, v) * ufl.dx - lambda_ * ufl.inner(ufl.grad(c), ufl.grad(v)) * ufl.dx
F = F0 + F1

# Create NonlinearProblem
problem = NonlinearProblem(F, u)

# Create Newton Solver
log.set_log_level(log.LogLevel.INFO)
solver = NewtonSolver(domain.comm, problem)
solver.convergence_criterion = "incremental"
solver.rtol = np.sqrt(np.finfo(default_real_type).eps) * 1e-2
ksp = solver.krylov_solver
opts = PETSc.Options()  # type: ignore
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "lu"
sys = PETSc.Sys()  # type: ignore
# For factorisation prefer MUMPS, then superlu_dist, then default
use_superlu = PETSc.IntType == np.int64  # or PETSc.ScalarType == np.complex64
if sys.hasExternalPackage("mumps") and not use_superlu:
    opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"
elif sys.hasExternalPackage("superlu_dist"):
    opts[f"{option_prefix}pc_factor_mat_solver_type"] = "superlu_dist"
ksp.setFromOptions()

# Post-process
# Save solution to file
t = 0.0
results_folder = Path("results")
results_folder.mkdir(exist_ok=True, parents=True)
filename = results_folder / "out_ch2p"
with io.XDMFFile(domain.comm, filename.with_suffix(".xdmf"), "w", io.XDMFFile.Encoding.ASCII) as xdmf:
    xdmf.write_mesh(domain)
# Get the sub-space for c
V0, dofs = ME.sub(0).collapse()

# Time-stepping
c = u.sub(0)
u0.x.array[:] = u.x.array
while t < T:
    t += dt
    r = solver.solve(u)
    print(f"Step {int(t / dt)}: num iterations: {r[0]}")
    u0.x.array[:] = u.x.array
    xdmf.write_function(c, t)
xdmf.close()

M=1.0, V=1.0, R=1.0, lambda=0.02
[2025-03-12 12:23:24.596] [info] Extract basic topology: 524288->524288
[2025-03-12 12:23:24.596] [info] Build local dual graph
[2025-03-12 12:23:24.596] [info] Build local part of mesh dual graph (mixed)
[2025-03-12 12:23:24.617] [info] GPS pseudo-diameter:(767) 131071-0
[2025-03-12 12:23:24.622] [info] Create topology (single cell type)
[2025-03-12 12:23:24.622] [info] Create topology (generalised)
[2025-03-12 12:23:24.625] [info] Computing communication graph edges (using NBX algorithm). Number of input edges: 1
[2025-03-12 12:23:24.625] [info] Finished graph edge discovery using NBX algorithm. Number of discovered edges 1
[2025-03-12 12:23:24.652] [info] Compute ghost indices
[2025-03-12 12:23:24.652] [info] Computing communication graph edges (using PCX algorithm). Number of input edges: 0
[2025-03-12 12:23:24.652] [info] Finished graph edge discovery using PCX algorithm. Number of discovered edges 0
[2025-03-12 12:23:24.683] [info] Computing commu